In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
# Import Library
import json
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
import random
nltk.download('wordnet')
nltk.download('omw-1.4')

# Baca data dari file CSV
data = pd.read_csv('data_clean.csv')  # Gantilah 'data.csv' dengan nama file CSV Anda

data

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,full_text,Labeling
0,hai sorry balesaku dapet beasiswa kipk kaa hhi...,Neutral
1,upnjog info beasiswa maba kipk gais sender ga ...,Neutral
2,aku bayar kuliah seok jd calon kipk tp ga teri...,Pros
3,beasiswa perintah kipk nih kasih dasar tara bi...,Cons
4,halo ka nanya bus beasiswa kipk nya gaa,Pros
...,...,...
7856,skrg wktunya awas skema kuat ekonomi rakyat kc...,Cons
7857,mgkn skema nya beda jakarta sdh kjp kjs g tump...,Pros
7858,inipmrnth blm skema plindungan sos thdp tkisla...,Cons
7859,inipmrnth blm skema plindungan sos thdp tkisla...,Cons


In [2]:
data['Labeling'] = data['Labeling'].replace({'D': 'Cons', 'Netrap': 'Neutral'})

In [3]:
# Preprocessing data
le = LabelEncoder()
data['Labeling'] = le.fit_transform(data['Labeling'])

In [4]:
data['Labeling'].unique()

array([1, 2, 0])

In [5]:
# Mengunduh dataset kata-kata stop dalam bahasa Inggris dari NLTK.
nltk.download('stopwords')

# Mengimpor modul stopwords dari nltk.corpus untuk mengakses kata-kata stop.
from nltk.corpus import stopwords

# Membuat sebuah list kosong
stop_words = []

# Melakukan iterasi melalui setiap kata dalam daftar kata-kata stop dalam bahasa Inggris.
for w in stopwords.words('indonesian'):
    # Menambahkan setiap kata stop ke dalam list.
    stop_words.append(w)

# Mencetak list yang berisi kata-kata stop dalam bahasa Inggris.
print(stop_words)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def get_synonyms(word):
    # Membuat sebuah set kosong untuk menyimpan sinonim.
    synonyms = set()

    # Melakukan iterasi melalui setiap synset (kumpulan kata dengan makna yang mirip)
    # dalam WordNet yang terkait dengan kata yang diberikan.
    for syn in wordnet.synsets(word):
        # Melakukan iterasi melalui setiap lemma (kata turunan) dalam synset.
        for l in syn.lemmas():
            # Mendapatkan kata sinonim, mengganti karakter khusus seperti _ dan - dengan spasi, mengubah huruf menjadi huruf kecil
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm']) # membersihkan karakter yang bukan huruf.

            # Menambahkan sinonim ke dalam set sinonim.
            synonyms.add(synonym)

    # Menghapus kata awal (kata yang diberikan) dari set sinonim, jika ada.
    if word in synonyms:
        synonyms.remove(word)

    # Mengembalikan list dari sinonim-sinonim yang ditemukan.
    return list(synonyms)


In [7]:
def synonym_replacement(words, n):
    # Memisahkan kata-kata dalam kalimat menjadi sebuah list dengan .split().
    words = words.split()

    # Membuat salinan list
    new_words = words.copy()
    # Membuat daftar kata-kata unik yang bukan merupakan kata-kata stop words.
    random_word_list = list(set([word for word in words if word not in stop_words]))
    # Mengacak urutan kata-kata unik.
    random.shuffle(random_word_list)

    # Inisialisasi variabel untuk menghitung berapa banyak kata yang sudah digantikan.
    num_replaced = 0

    # Melakukan iterasi melalui daftar kata-kata unik yang telah diacak.
    for random_word in random_word_list:
        # Mendapatkan sinonim-sinonim dari kata acak dengan fungsi get_synonyms() yang kita buat sebelumnya.
        synonyms = get_synonyms(random_word)

        # Jika ada setidaknya satu sinonim yang ditemukan.
        if len(synonyms) >= 1:
            # Memilih salah satu sinonim secara acak.
            synonym = random.choice(list(synonyms))

            # Mengganti kata acak dengan sinonimnya dalam new_words.
            new_words = [synonym if word == random_word else word for word in new_words]

            # Menambah jumlah kata yang sudah digantikan.
            num_replaced += 1

        # Jika jumlah kata yang sudah digantikan mencapai n, maka berhenti.
        if num_replaced >= n:
            break

    # Menggabungkan kata-kata dalam new_words menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    # Mengembalikan nilai dari fungsi menjadi sebuah kalimat dari kata-kata.
    return sentence


In [8]:
def random_deletion(words, p):

    words = words.split()

    # Jika kalimat hanya terdiri dari satu kata, maka kalimat tersebut dikembalikan tanpa perubahan.
    if len(words) == 1:
        return words

    # Membuat list baru untuk kata-kata yang akan dipertahankan.
    new_words = []

    # Melakukan iterasi melalui kata-kata dalam kalimat.
    for word in words:
        # Menghasilkan angka acak antara 0 dan 1.
        r = random.uniform(0, 1)

        # Jika nilai acak r lebih besar dari p (probabilitas), maka kata tersebut akan dipertahankan dengan menambahkannya ke dalam list new_words.
        if r > p:
            new_words.append(word)

    # Jika tidak ada kata yang dipertahankan (semua kata dihapus), maka satu kata acak dipilih untuk tetap ada dalam kalimat.
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    # Menggabungkan kata-kata yang dipertahankan dalam list menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    return sentence


In [9]:
def swap_word(new_words):
    # Menghasilkan dua indeks acak dalam range dari 0 hingga panjang new_words - 1.
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    # Melakukan loop sampai indeks kedua tidak sama dengan indeks pertama.
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1

        # Jika counter melebihi 3, maka kembalikan new_words tanpa perubahan.
        if counter > 3:
            return new_words

    # Menukar kata di indeks pertama dengan kata di indeks kedua dalam new_words.
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]

    # Mengembalikan new_words setelah kata-kata ditukar.
    return new_words


In [10]:
def random_swap(words, n):

    words = words.split()
    # Membuat salinan list kata-kata yang akan dimodifikasi.
    new_words = words.copy()

    # Melakukan pertukaran kata-kata sebanyak n kali.
    for _ in range(n):
        new_words = swap_word(new_words)  # Memanggil fungsi swap_word() yang sudah dibuat untuk menukar kata-kata.

    # Menggabungkan kata-kata yang sudah ditukar kembali menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    # Mengembalikan kalimat setelah semua pertukaran kata-kata selesai.
    return sentence

In [11]:
# Menampilkan data pada dataframe indeks 0
words = data['full_text'][0]
print(words)

hai sorry balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa


In [12]:
# Penerapan Synonym Replacement dengan fungsi synonym_replacement(words,n)
for n in range(5):
    print(f" Example of Synonym Replacement: {synonym_replacement(words,n)}")

 Example of Synonym Replacement: hai sorry balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa
 Example of Synonym Replacement: hai blue balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa
 Example of Synonym Replacement: hai no good balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa
 Example of Synonym Replacement: hai dismal balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa
 Example of Synonym Replacement: hai sad balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa


In [13]:
# Penerapan Random Deletion dengan fungsi random_deletion(words,n)
print("Original:", words)
print('\nRandom Deletion:')
print(random_deletion(words,0.2))
print(random_deletion(words,0.3))
print(random_deletion(words,0.4))

Original: hai sorry balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa

Random Deletion:
hai sorry balesaku dapet kipk kaa hhidi kampus pengumumannyaa
sorry balesaku beasiswa kipk kaa hhidi kampus pengumumannyaa
hai sorry balesaku kipk kaa kampus pengumumannyaa


In [14]:
# Penerapan Random Swap dengan fungsi random_swap(words,n)
print("Original:", words)
print('\nRandom Swap:')
print(random_swap(words,1))
print(random_swap(words,2))
print(random_swap(words,3))

Original: hai sorry balesaku dapet beasiswa kipk kaa hhidi kampus pengumumannyaa

Random Swap:
hai sorry balesaku kaa beasiswa kipk dapet hhidi kampus pengumumannyaa
hai sorry balesaku hhidi kampus kipk kaa dapet beasiswa pengumumannyaa
kaa sorry balesaku pengumumannyaa beasiswa dapet hai hhidi kampus kipk


In [15]:
# Augmentasi data
augmented_text = []

for n in range(len(data)):
    word = data['full_text'][n]

    synonym_replaced = synonym_replacement(word, 3)
    augmented_text.append((synonym_replaced, data['Labeling'][n]))

    random_swapped = random_swap(word, 3)
    augmented_text.append((random_swapped, data['Labeling'][n]))

    random_deleted = random_deletion(word, 0.2)
    augmented_text.append((random_deleted, data['Labeling'][n]))

# Mengubah list di atas menjadi DataFrame
augmented_df = pd.DataFrame(augmented_text, columns=['Text', 'Label'])
augmented_df

,Text,Label
0,hai good for naught balesaku dapet beasiswa ki...,1
1,hai sorry dapet kampus beasiswa kipk pengumuma...,1
2,balesaku beasiswa kipk kaa hhidi kampus pengum...,1
3,upnjog information beasiswa maba kipk gais sen...,1
4,info upnjog beasiswa tolong kipk gais sender g...,1
...,...,...
23578,asuransi blm komersil skema sos thdp tkislain ...,0
23579,inipmrnth skema plindungan thdp tkislain komer...,0
23580,undang sleep kalo kan decigram skema childlike...,1
23581,undang uu kalo bikin dg skema simple lahhh alu...,1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(augmented_df['Text'], augmented_df['Label'], test_size=0.2, random_state=0)

In [17]:
# Tokenisasi teks
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [18]:
# Padding sequence agar memiliki panjang yang sama
max_len = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post', truncating='post')


In [19]:
# Membuat model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Menggunakan softmax karena ada tiga kelas


In [20]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [21]:
# Menghentikan pelatihan jika tidak ada peningkatan dalam validation loss
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Melatih model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test), callbacks=[early_stopping])


Epoch 1/10
590/590 [==============================] - 32s 53ms/step - loss: 0.8810 - accuracy: 0.5982 - val_loss: 0.7407 - val_accuracy: 0.6824
Epoch 2/10
590/590 [==============================] - 33s 55ms/step - loss: 0.3902 - accuracy: 0.8517 - val_loss: 0.5227 - val_accuracy: 0.8045
Epoch 3/10
590/590 [==============================] - 34s 58ms/step - loss: 0.1413 - accuracy: 0.9517 - val_loss: 0.5636 - val_accuracy: 0.8107
Epoch 4/10
590/590 [==============================] - 35s 59ms/step - loss: 0.0694 - accuracy: 0.9772 - val_loss: 0.6457 - val_accuracy: 0.8056
Epoch 5/10
590/590 [==============================] - 36s 61ms/step - loss: 0.0451 - accuracy: 0.9851 - val_loss: 0.7389 - val_accuracy: 0.8084


In [22]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Akurasi:", accuracy)

148/148 [==============================] - 1s 5ms/step - loss: 0.5227 - accuracy: 0.8045
Akurasi: 0.804536759853363


In [23]:
y_pred_prob = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_prob, axis=1)

148/148 [==============================] - 1s 4ms/step


In [24]:
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8045367818528726
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.83      0.82      1783
           1       0.83      0.84      0.84      1806
           2       0.74      0.70      0.72      1128

    accuracy                           0.80      4717
   macro avg       0.80      0.79      0.79      4717
weighted avg       0.80      0.80      0.80      4717

